In [1042]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
import pickle

In [1043]:
# Loading the dataset
df = pd.read_csv(r'Churn_Modelling.csv')
df.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [1044]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [1045]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [1046]:
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [1047]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [1048]:
df.drop(columns=['Surname', 'CustomerId','RowNumber'], inplace=True)
df.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [1049]:
# when we write fit the what does happen assign value to all class
# when we write transform then in whole dataset whereever that class available 
# assign that value what learned earlier
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])

In [1050]:
# we can't use labelencoder for Geography beacuse model can assume as ranking. which will create problem.
oe = OneHotEncoder(sparse_output=False)
encoded_geo = oe.fit_transform(df[['Geography']])
oe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [1051]:
geo_encoded_df = pd.DataFrame(data=encoded_geo, columns=['Geography_France', 'Geography_Germany', 'Geography_Spain'])

In [1052]:
df.drop(columns=['Geography'], inplace=True)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,42,2,0.00,1,1,1,101348.88,1
1,608,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,39,1,0.00,2,0,0,93826.63,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0


In [1053]:
final_df = pd.concat([df,geo_encoded_df], axis=1)
final_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [1054]:
with open('le.pkl','wb') as file:
    pickle.dump(le,file)

with open('ohe.pkl','wb') as file:
    pickle.dump(oe,file)


In [1055]:
# Independent and dependent features
x = final_df.drop(columns=['Exited'])
y =final_df['Exited']

In [1056]:
# train-test split
Xtrain, Xtest, ytrain, ytest = train_test_split(x,y,test_size=0.25,random_state=42)
# Standardization why not normalization.
# Why MinMaxScaler for TensorFlow:
# Neural networks are sensitive to the scale of input features.
# Activations like ReLU, sigmoid, and tanh perform better when inputs are within a normalized range (typically [0, 1]).
# MinMaxScaler helps speed up convergence during training and improves numerical stability.
# ------------------
# ⚠️ Important Notes:
# Real-world data is rarely perfectly normal.
# For deep learning, MinMaxScaler is usually better even if data is normal, because of how neural nets behave with values outside [0, 1].
mm = MinMaxScaler()
X_train_scaled = mm.fit_transform(Xtrain)
X_test_scaled = mm.transform(Xtest)

In [1057]:
with open('mm.pkl','wb') as file:
    pickle.dump(mm,file)

In [1058]:
X_train_scaled.shape

(7500, 12)

In [1059]:
# ANN
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Dense

In [1060]:
# Build a Artificial Neural network.
models = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1,activation='sigmoid')
])

In [1061]:
# forward and backward propgation
import tensorflow
losses = tensorflow.keras.losses.BinaryCrossentropy()
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
models.compile(loss=losses, optimizer=opt, metrics=['accuracy'])

In [1062]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
import os
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tb = TensorBoard(log_dir=logdir, histogram_freq=1)

In [1063]:
es = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

In [1064]:
# Train the model
models.fit(
    X_train_scaled, ytrain,
    validation_data = (X_test_scaled, ytest),
    epochs = 100,
    callbacks = [tb, es]
    
)

Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7982 - loss: 0.4781 - val_accuracy: 0.8196 - val_loss: 0.4129
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step - accuracy: 0.8223 - loss: 0.4105 - val_accuracy: 0.8392 - val_loss: 0.3874
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step - accuracy: 0.8470 - loss: 0.3628 - val_accuracy: 0.8488 - val_loss: 0.3697
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 650us/step - accuracy: 0.8437 - loss: 0.3713 - val_accuracy: 0.8556 - val_loss: 0.3501
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 645us/step - accuracy: 0.8478 - loss: 0.3534 - val_accuracy: 0.8600 - val_loss: 0.3470
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step - accuracy: 0.8531 - loss: 0.3596 - val_accuracy: 0.8560 - val_loss: 0.3526
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - accuracy: 0.8518 - loss: 0.3521 - val_accuracy: 0.8544 - val_loss: 0.3516
Epoch 8/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 677us/step - accuracy: 0.8556 - loss: 0.3

In [1065]:
models.save('models.keras')
models.save('models.h5')

In [1066]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [1067]:
final_df.head(1)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.0,1,1,1,101348.88,1,1.0,0.0,0.0


In [1068]:
# Predictions
input = {
    'CreditScore' : 750,
    'Gender': 'Male',
    'Age': 30,
    'Tenure' : 3,
    'Balance' : 2000,
    'NumOfProducts' : 3,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 151348.88,
    'Geography': 'Spain'
}

In [1069]:
input_df = pd.DataFrame(data=[input])
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,750,Male,30,3,2000,3,1,1,151348.88,Spain


In [1070]:
with open('le.pkl', 'rb') as file:
    lablel_encoder_gender_load = pickle.load(file)

with open('ohe.pkl', 'rb') as file:
    one_geography_load = pickle.load(file)

with open('mm.pkl', 'rb') as file:
    std_encoder_load = pickle.load(file)

model_pred = load_model('models.h5')


In [1071]:
input_df['Gender'] = lablel_encoder_gender_load.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,750,1,30,3,2000,3,1,1,151348.88,Spain


In [1072]:
geo_features = one_geography_load.transform(input_df[['Geography']])
# oe.get_feature_names_out(['Geography'])
geo_features_df = pd.DataFrame(data=geo_features, columns=['Geography_France', 'Geography_Germany', 'Geography_Spain'])
geo_features_df

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [1073]:
input_df.drop(columns=['Geography'], inplace=True)

In [1074]:

res_df = pd.concat([input_df.reset_index(drop=True), geo_features_df], axis=1)
res_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,750,1,30,3,2000,3,1,1,151348.88,0.0,0.0,1.0


In [1075]:
input_scaled = std_encoder_load.transform(res_df)

In [1078]:
pred = model_pred.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [1079]:
result = pred[0][0]

In [1081]:
result
if(result >0.5) : 
    print('customer not churn')
else:
    print('customer churn')

customer churn


In [1076]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 67354), started 1 day, 5:10:53 ago. (Use '!kill 67354' to kill it.)